In [7]:
import pandas as pd
import numpy as np
import pymssql
import warnings
warnings.filterwarnings("ignore")
from gensim.models import Word2Vec
import pickle

### Класс подключения к БД

In [8]:
from dataclasses import dataclass
from sqlalchemy import create_engine, event, DateTime, Column, String, MetaData, Integer, \
    Binary, PrimaryKeyConstraint, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, Session
from contextlib import contextmanager
from urllib.parse import quote_plus

@dataclass(frozen=True)
class DbConfig:
    db_server = "10.252.4.116"
    pwd = "BDP4b8f6"
    uid = "ext-E.Stepanova"
    db_name = "DWH_Globus"
    driver = r"{ODBC Driver 17 for SQL Server}"
    params = quote_plus(
        'DRIVER={DRIVER};SERVER={DB_SERVER};DATABASE={DB_NAME};UID={UID};PWD={PWD}'.format(
            DB_SERVER=db_server, DB_NAME=db_name, UID=uid, PWD=pwd, DRIVER=driver
        ))
    print(params)
    conn_str = 'mssql+pyodbc:///?odbc_connect={}'.format(params)


class Db:
    def __init__(self):
        self._db_conf = DbConfig()
        self.engine = None

    def create_engine(self):
        if self.engine is None:
            self.engine = create_engine(self._db_conf.conn_str)

        @event.listens_for(self.engine, 'before_cursor_execute')
        def receive_before_cursor_execute(conn, cursor, statement, params, context, executemany):
            if executemany:
                cursor.fast_executemany = True
                cursor.commit()

    @contextmanager
    def open_session(self):
        """Provide a transactional scope around a series of operations."""
        session: Session = sessionmaker(bind=self.engine)()
        try:
            yield session
            session.commit()
        except:
            session.rollback()
            raise
        finally:
            session.close()

DRIVER%3D%7BODBC+Driver+17+for+SQL+Server%7D%3BSERVER%3D10.252.4.116%3BDATABASE%3DDWH_Globus%3BUID%3Dext-E.Stepanova%3BPWD%3DBDP4b8f6


### Загружаем данные без пластиковых пакетов

In [3]:
str_sql_query_visits ="""select distinct
                m.MEMS_ID, -- ID пользователя
                s.BonDatum, s.GBNr, -- ID визита
                s.MATNR as product_id -- ID товара
                from DWH_Globus.CDA_EXT.IDC_S s with (nolock)
                left join DWH_Globus.crm_std.Members m with (nolock)
                               on s.LoyKarte = m.CARD_EXT_NUM
                where m.ZeileBisDatum = '2999-12-31' and s.MATNR not in
                (154481, 457553,  64987, 157852, 139189, 131994,  86591,
       391803, 392408, 397429,  38565, 157849,  11789,  80689,
       446888, 392407,  11786,  80692, 195105, 123023,  46022,
       457515,  80695, 121389, 509184, 480292,  11785, 204279,
        80690,  11788, 457552, 123731,  80691)
                               and s.Datum between '2019-03-01' and '2019-09-01' -- смотрим за пол года
                               and Kode1=1 
"""
db = Db()
db.create_engine()
chunks = []
for chunk in pd.read_sql(str_sql_query_visits, con=db.engine, chunksize = 10**4):
    chunks.append(chunk)
df_visits= pd.concat(chunks)


### Чистим данные от дубликатов и NULL

In [4]:
df_visits=df_visits.drop_duplicates(keep='first')
df_visits=df_visits.dropna()

### Меняем тип переменной product_id на строку

In [5]:
df_visits['product_id'] = df_visits['product_id'].astype('int64').astype('str')


### Делаем таблицу транзакций (списков product_id)

In [6]:
transactions=df_visits.groupby(['BonDatum','MEMS_ID', 'GBNr'])['product_id'].apply(list).reset_index(name='transactions')

### Чистим транзакции от единичных покупок

In [7]:
transactions['len']=transactions['transactions'].apply(len)

In [8]:
transactions=transactions.drop(transactions[transactions['len']<2].index)

### Выбираем окно для модели 

In [9]:
window=transactions['len'].quantile(q=0.95, interpolation='nearest')//2

In [ ]:
## Make model, pack, serch substitutes

### Делаем список транзакций, который пойдет на вход модели 

In [11]:
my_list=[]
for i in range(len(transactions)):
        my_list.append(transactions['transactions'].values[i])

### Обучаем и запаковываем модель

In [12]:
model_cbow = Word2Vec(my_list,size=300,window=window,min_count=1,workers=8, negative=3)
model_cbow.init_sims(replace=True)
with open('Woerd2vec_cbow1302.pkl', 'wb') as fid:
    pickle.dump(model_cbow, fid)

### Распаковка модели

In [13]:
best_model = pickle.load(open("Woerd2vec_cbow1302.pkl", "rb"))


### Создаем массив всех уникальных product_id

In [14]:
products=df_visits['product_id'].unique() 

### Записываем заменители для всех products_id в файл '/mnt/cda/subs_1302.csv'

In [15]:

df_subs=pd.DataFrame(columns= ['subst_id', 'cos', 'product_id'])
df_subs.to_csv('/mnt/cda/subs_1302.csv', encoding='cp1251', sep=';', 
                    header = ['subst_id', 'cos', 'product_id'])
for j in range(len(products)):
    if products[j] in best_model.wv.vocab:
               result=pd.DataFrame(np.asarray(best_model.most_similar(products[j], topn=50) ), columns=['subst_id', 'cos'])
               result['product_id']=products[j]
               result.to_csv('/mnt/cda/subs_1302.csv', mode='a', encoding='cp1251', sep=';', 
                    header = False)


## Фильтруем заменители на разных товарных уровнях

### Загружаем заменители из файла '/mnt/cda/subs_1302.csv'

In [9]:
substitutes=pd.read_csv('/mnt/cda/subs_1302.csv', encoding='cp1251', sep=';').drop('Unnamed: 0', axis=1)

### Загружаем имена и группы товаров

In [10]:
str_sql_query_names="""select distinct m.MATNR as product_id,s.Level3_Name,s.Level3_ID, s.Level4_ID,
s.product_group_40,s.Name_product_group_40, s.Level4_Name, x.WgRU, x.Wg, x.AEbene, 
x.AEbeneRU, x.BEbene, x.BEbeneRU,x.OberWGRU, x.OberWG, x.Kategorie, x.KategorieRU,
s.product_name from DWH_Globus.[cda_marts].[AGG_PRODUCT_ML] s with (nolock)
left join DWH_Globus.rtl_std.MARAGeschichte m with (nolock) on s.product_id=m.MATNR
inner join DWH_Globus.rtl_std.WarengruppeHierarchie x with (nolock) on m.MATKL = x.Wg
where m.ZeileBisDatum = '29991231' and s.load_dt = '20200201'"""
db = Db()
db.create_engine()
chunks = []
for chunk in pd.read_sql(str_sql_query_names, con=db.engine, chunksize = 10**5):
    chunks.append(chunk)
df_names= pd.concat(chunks)

### Соединяем имена и группы с id товаров

In [11]:
df_all=pd.merge(substitutes, df_names, on='product_id', how='left')

In [12]:
df_names['Level3_Name_subs']=df_names['Level3_Name']
df_names['Level4_Name_subs']=df_names['Level4_Name']
df_names['subst_id']=df_names['product_id']
df_names['subst_name']=df_names['product_name']
df_names['WgRU_subs']=df_names['WgRU']
df_names['Level3_ID_subs']=df_names['Level3_ID']
df_names['Level4_ID_subs']=df_names['Level4_ID']
df_names['product_group_40_subs']=df_names['product_group_40']
df_names['Name_product_group_40_subs']=df_names['Name_product_group_40']
df_names['Wg_subs']=df_names['Wg']
df_names['AEbene_subs']=df_names['AEbene']
df_names['AEbeneRU_subs']=df_names['AEbeneRU']
df_names['BEbene_subs']=df_names['BEbene']
df_names['BEbeneRU_subs']=df_names['BEbeneRU']
df_names['OberWGRU_subs']=df_names['OberWGRU']
df_names['OberWG_subs']=df_names['OberWG']
df_names['Kategorie_subs']=df_names['Kategorie']
df_names['KategorieRU_subs']=df_names['KategorieRU']

In [13]:
df_all=df_all.merge(df_names[['Level3_Name_subs', 'Level4_Name_subs', 'subst_id', 'subst_name', 'WgRU_subs',
       'Level3_ID_subs', 'Level4_ID_subs', 'product_group_40_subs',
       'Name_product_group_40_subs', 'Wg_subs', 'AEbene_subs', 'AEbeneRU_subs', 'BEbene_subs',
       'BEbeneRU_subs', 'OberWGRU_subs', 'OberWG_subs',
       'Kategorie_subs', 'KategorieRU_subs']], on='subst_id', how='left')

### Создаем индексы для фильтрации

In [14]:
df_all['GRT_ind']=(df_all['Wg'] == df_all['Wg_subs']).astype(int)
df_all['Lvl4_ind']=(df_all['Level4_ID_subs']==df_all['Level4_ID']).astype(int)
df_all['Lvl3_ind']=(df_all['Level3_ID_subs']==df_all['Level3_ID']).astype(int)
df_all['A_ind']=(df_all['AEbene_subs']==df_all['AEbene']).astype(int)
df_all['B_ind']=(df_all['BEbene_subs']==df_all['BEbene']).astype(int)
df_all['group_ind']=(df_all['product_group_40_subs']==df_all['product_group_40']).astype(int)
df_all['Ober_ind']=(df_all['OberWG_subs']==df_all['OberWG']).astype(int)
df_all['Kat_ind']=(df_all['Kategorie_subs']==df_all['Kategorie']).astype(int)

In [15]:
df_all.to_csv('/mnt/cda/subs_1302_name.csv', encoding='cp1251', sep=';')

### Загружаем файл с уровнями фильтрации

In [16]:
filtr=pd.read_excel('/mnt/cda/Limits.xlsx')

### Соединяем уровни фильтрации с данными заменителей

In [17]:
df_filter=pd.merge(df_all, filtr, on='Wg', how='left')

### Удаляем ненужные product_id

In [18]:
df_filter=df_filter.drop(df_filter[df_filter['Limit1']=='NOT FOUND'].index)
df_filter=df_filter.drop(df_filter[(df_filter['Limit1']=='BEbeneRU')&(df_filter['B_ind']==0)].index)
df_filter=df_filter.drop(df_filter[(df_filter['Limit1']=='AEbeneRU')&(df_filter['A_ind']==0)].index)
df_filter=df_filter.drop(df_filter[(df_filter['Limit1']=='OberWGRU')&(df_filter['Ober_ind']==0)].index)
df_filter=df_filter.drop(df_filter[(df_filter['Limit1']=='KategorieRU')&(df_filter['Kat_ind']==0)].index)
df_filter=df_filter.drop(df_filter[(df_filter['Limit1']=='L4')&(df_filter['Lvl4_ind']==0)].index)
df_filter=df_filter.drop(df_filter[(df_filter['Limit1']=='WG')&(df_filter['GRT_ind']==0)].index)
df_filter=df_filter.drop(df_filter[(df_filter['Limit1']=='L3')&(df_filter['Lvl3_ind']==0)].index)

### Сохраняем результирующий файл

In [19]:
df_filter.to_csv('/mnt/cda/subs_1302_filtr.csv', encoding='cp1251', sep=';')